In [1]:
%pwd

'/Users/hale/PycharmProjects/ClaimGPT250203'

In [2]:
%ls

LICENSE          claim_gpt.ipynb  markdown/        shared/
README.md        corpus_base/     math_gpt_output/
claim_gpt/       internal/        notebooks/


In [3]:
import sys
import os

import torch
import random
from pathlib import Path
from dataclasses import dataclass
from sklearn.model_selection import train_test_split

import re

In [4]:
from claim_gpt.create_files.create_files import create_files
from claim_gpt.create_model.create_model import create_model
from claim_gpt.train_model.train_model import train_model
from claim_gpt.validate_model.validate_model import validate_model

from shared import Encoder
from shared import load_model

In [5]:
for path in sys.path:
    print(path)

/opt/anaconda3/envs/python311/lib/python311.zip
/opt/anaconda3/envs/python311/lib/python3.11
/opt/anaconda3/envs/python311/lib/python3.11/lib-dynload

/opt/anaconda3/envs/python311/lib/python3.11/site-packages
/opt/anaconda3/envs/python311/lib/python3.11/site-packages/setuptools/_vendor


# Settings

In [6]:
@dataclass
class Settings():
    block_size: int = 150 # max dictum size
    limit_count = 1000 * 50 # max for corpus FIXME: not quite right

settings = Settings()

# output_folder_path

In [7]:
output_folder_path = Path('math_gpt_output/main_claim/').resolve()

# Create corpus

In [9]:
%%time
# create claim corpus
block_size = settings.block_size
limit_count = settings.limit_count
if output_folder_path.joinpath('corpus/corpus.txt').exists():
    corpus_file_path = output_folder_path.joinpath('corpus/corpus.txt')
else:
    corpus_file_path = create_files(output_folder_path=output_folder_path, block_size=block_size, limit_count=limit_count)
    with open(corpus_file_path, 'r') as file:
        corpus_statements = file.read().splitlines()
        X_train, X_test = train_test_split(corpus_statements, test_size=0.2, random_state=42)
        with open(corpus_file_path.parent.joinpath("train_corpus.txt"), "w") as outfile:
            outfile.write("\n".join(X_train))
        with open(corpus_file_path.parent.joinpath("test_corpus.txt"), "w") as outfile:
            outfile.write("\n".join(X_test))
print(f'corpus_file_path={corpus_file_path}')

10000: equsalhw
20000: $p
30000: dmsnn0
40000: suppssof1
50000: ac4c.1
60000: xaddge0
70000: $e
80000: $p
90000: ${
100000: $e
110000: $e
120000: m2detleiblem2.g
130000: $e
140000: dyaddisjlem
150000: $}
160000: $e
170000: $e
180000: $}
190000: $p
200000: $d
210000: $}
220000: $e
230000: $e
240000: $e
250000: $d
260000: ${
270000: infrpge.xph
280000: $e
290000: $e
300000: $d
['$(', 'End', '$[', 'set-mbox.mm', '$]', '$)']

=== idi ===
conclusion: $p idi |- ph $= idi.1 $.

=== a1ii ===
conclusion: $p a1ii |- ph $= a1ii.1 $.

=== mp2 ===
conclusion: $p mp2 |- ch $= wps wch mp2.2 wph wps wch wi mp2.1 mp2.3 ax-mp ax-mp $.
utterance=['claim', 'ax-mp', ['|- ph', '|- ( ph -> ( ps -> ch ) )'], '|- ( ps -> ch )']
utterance=['claim', 'ax-mp', ['|- ps', '|- ( ps -> ch )'], '|- ch']

=== mp2b ===
conclusion: $p mp2b |- ch $= wps wch wph wps mp2b.1 mp2b.2 ax-mp mp2b.3 ax-mp $.
utterance=['claim', 'ax-mp', ['|- ph', '|- ( ph -> ps )'], '|- ps']
utterance=['claim', 'ax-mp', ['|- ps', '|- ( ps -> ch )'

# Create Encoder

In [10]:
encoder = Encoder.load_from_json(corpus_folder_path=corpus_file_path.parent)

# Create model

In [11]:
def set_up_model(corpus_file_path: Path, model_info: (str, int, int), output_folder_path: Path) -> Path:
    model_name, n_head, n_layer = model_info
    model_folder_path = output_folder_path.joinpath('models/').resolve()
    model_file_path = model_folder_path.joinpath(model_name).resolve()
    create_model(model_file_path=model_file_path, corpus_file_path=corpus_file_path, n_head=n_head, n_layer=n_layer)
    return model_file_path

In [12]:
model_info = ('model/model.pt', 10, 10) # model_name, n_head, n_layer
model_file_path = set_up_model(corpus_file_path=corpus_file_path, model_info=model_info, output_folder_path=output_folder_path)
print(f'model_file_path={model_file_path}')
device = 'cuda' if torch.cuda.is_available() else 'cpu'

torch.set_default_device(device)

print(f'device={device}')
print(f'loading model and optimizer from checkpoint={model_file_path}')
model, optimizer = load_model(model_checkpoint_path=model_file_path, device=device, encoder=encoder)
print(f'model.device={model.device}')

Start create model and optimizer
create_model: model_checkpoint_path=/Users/hale/PycharmProjects/ClaimGPT250203/math_gpt_output/main_claim/models/model/model.pt
create_encoder
vocab_size=666
create model: block_size=150 device=cpu
121.584666 M parameters
create a PyTorch optimizer
model created at path=/Users/hale/PycharmProjects/ClaimGPT250203/math_gpt_output/main_claim/models/model/model.pt
Done
model_file_path=/Users/hale/PycharmProjects/ClaimGPT250203/math_gpt_output/main_claim/models/model/model.pt
device=cpu
loading model and optimizer from checkpoint=/Users/hale/PycharmProjects/ClaimGPT250203/math_gpt_output/main_claim/models/model/model.pt
model.device=cpu


# Train model

In [13]:
%%time
if model.device == 'cpu':
    max_train_epochs = 10 * 1 * 1 * 1 # cpu: 10 * 10 * 5 is about 80 minutes
else:
    max_train_epochs = 100 * 10 * 10 * 1 # gpu fast: 100 * 10 * 1 is about 5 minutes
train_corpus_file_path = corpus_file_path.parent.joinpath('train_corpus.txt')
train_model(model, optimizer, max_train_epochs=max_train_epochs, corpus_file_path=train_corpus_file_path, model_file_path=model_file_path)
print('Done')

vocab_size=666
corpus_file_path=/Users/hale/PycharmProjects/ClaimGPT250203/math_gpt_output/main_claim/corpus/train_corpus.txt
corpus_statement_count=40000
#encoded_train_statements=40000
=== train and evalate model ===
epoch=0; step=0; n_head=10; n_layer=10
train_batch_size=10
max_train_epochs=10 eval_interval=1
block_size=150 n_layer=10 learning_rate=0.0001 device=cpu
#train_dataset=40000
       0. train loss 6.6422
       1. train loss 4.9982
       2. train loss 2.4167
       3. train loss 3.2872
       4. train loss 2.0177
       5. train loss 1.9973
       6. train loss 1.9221
       7. train loss 3.3926
       8. train loss 1.3208
       9. train loss 1.4005
elapsed_time=1.38 minutes
epoch=10; step=100
saving model: epoch=10 model_checkpoint_path=/Users/hale/PycharmProjects/ClaimGPT250203/math_gpt_output/main_claim/models/model/model.pt
Done
CPU times: user 14min 52s, sys: 5min 56s, total: 20min 48s
Wall time: 1min 27s


# Validate model

In [14]:
%%time
if model.device == 'cpu':
    max_examples=10 * 1 # better the model, the more examples one can do quicker
else:
    max_examples=100 * 1 # better the model, the more examples one can do quicker
max_print_error = 10
max_print_ok = 3
if max_examples > 0:
    print(f'--- validate model (count={max_examples}) ---')
    test_corpus_file_path = corpus_file_path.parent.joinpath('test_corpus.txt')
    validate_model(model=model, max_examples=max_examples, max_print_error=max_print_error, max_print_ok=max_print_ok, corpus_file_path=test_corpus_file_path)
print('Done')

--- validate model (count=10) ---
corpus_file_path=/Users/hale/PycharmProjects/ClaimGPT250203/math_gpt_output/main_claim/corpus/test_corpus.txt
corpus_statement_count=10000
epoch=10; step=100; n_head=10; n_layer=10
#val_statements=2000
=== start ===
max_examples=10
===== Example 1 error:1 =====
<|start_claim|> <|given|> |- ( S e. ( SubGrp ` G ) -> H e. Grp ) <|given|> |- ( H e. Grp -> ( 0g ` H ) e. ( Base ` H ) ) <|conclude|> |- ( S e. ( SubGrp ` G ) -> ( 0g ` H ) e. ( Base ` H ) ) <|end_claim|>
<|start_claim|> <|given|> |- ( S e. ( SubGrp ` G ) -> H e. Grp ) <|given|> |- ( H e. Grp -> ( 0g ` H ) e. ( Base ` H ) ) <|conclude|> ) inr /\ ` ( +Q <|given|> e.
===== Example 2 error:1 =====
<|start_claim|> <|given|> |- ( ( ph /\ x e. T ) -> ( F ` x ) e. ( 0 ..^ N ) ) <|given|> |- ( ( F ` x ) e. ( 0 ..^ N ) -> ( F ` x ) e. ZZ ) <|conclude|> |- ( ( ph /\ x e. T ) -> ( F ` x ) e. ZZ ) <|end_claim|>
<|start_claim|> <|given|> |- ( ( ph /\ x e. T ) -> ( F ` x ) e. ( 0 ..^ N ) ) <|given|> |- ( ( F 

# Simulate Deployment

Note: not the best coding

Note:
We selected prefixes = ['ax-mp ', 'mp2 ', 'mp2b ', 'mpd ', 'syl '] in create_corpus.py.

But ax_mp appears most often.

TODO: should include these prefixes in the 'corpus' to be able to better filter.

TODO: add some more prefixes ???

TODO: print how many cases are there for each prefix.

In [15]:
def print_row(row):
    x = re.split(r"(?=<\|start_claim\|> | <\|given\|> | <\|conclude\|> | <\|end_claim\|>)", row)
    for item in x:
        print(item)

In [25]:
from shared import get_encoded_statement
from shared import generate_predicted_dictum

def simulate_deployment(test_corpus_statements):
    statement = None
    prompt = ''
    reply = 'error'
    conclusion_token = encoder.stoi['<|conclude|>']
    for _ in range(1):
        random_statement = random.choice(test_corpus_statements)
        encoded_val_statement = get_encoded_statement(random_statement, encoder, block_size)
        val_statement = encoder.decode(encoded_val_statement)
        random_prompt = val_statement.split(' <|conclude|> ')[0] + ' <|conclude|>'
        terminal_token = '<|end_claim|>'
        predicted_dictum = generate_predicted_dictum(prompt=random_prompt, terminal_token=terminal_token, model=model)
        statement = random_statement
        prompt = random_prompt
        if random_statement == predicted_dictum:
            reply = predicted_dictum.split(' <|conclude|> ')[1]
            break
    return statement, prompt, reply

In [26]:
model.eval()
test_corpus_file_path = corpus_file_path.parent.joinpath('test_corpus.txt')
print(f'test_corpus_file_path={test_corpus_file_path}')
with open(test_corpus_file_path, 'r') as file:
    test_corpus_statements = file.read().splitlines()
print(f'test_corpus_statement_count={len(test_corpus_statements)}')

mp_test_corpus_statements = [x for x in test_corpus_statements if x.count('given') > 2]
print(f'mp_test_corpus_statements={len(mp_test_corpus_statements)}')

test_corpus_file_path=/Users/hale/PycharmProjects/ClaimGPT250203/math_gpt_output/main_claim/corpus/test_corpus.txt
test_corpus_statement_count=10000
mp_test_corpus_statements=120


In [27]:
# Rerun this cell to simulate deployment
statement, prompt, reply = simulate_deployment(test_corpus_statements)
print_row('You:' + prompt + ' ')
print_row('Model:\n' + reply + ' ')

You:
<|start_claim|>
 <|given|> |- ( k e. ( ZZ \ A ) -> -. k e. A )
 <|given|> |- ( -. k e. A -> if ( k e. A , B , 1 ) = 1 )
 <|conclude|> 
Model:
error 


In [28]:
# Rerun this cell to simulate deployment for prefixes mp2 and mp2b
statement, prompt, reply = simulate_deployment(mp_test_corpus_statements)
print_row('You:' + prompt + ' ')
print_row('Model:\n' + reply + ' ')

You:
<|start_claim|>
 <|given|> |- _pi e. RR+
 <|given|> |- ( _pi e. RR+ -> ( _pi / 2 ) e. RR+ )
 <|given|> |- ( ( _pi / 2 ) e. RR+ -> 0 < ( _pi / 2 ) )
 <|conclude|> 
Model:
error 
